In [ ]:
from plugins.selenium import Selenium
from selenium.webdriver.common.by import By

import pandas as pd
import string

In [ ]:
webdriver = Selenium()
webdriver.start()
webdriver.go_to_url("https://inanyplace.blogspot.com/2017/01/lista-de-estados-brasileiros-sigla-estado-capital-e-regiao.html")

In [ ]:
driver = webdriver.get_driver()

In [ ]:
table = driver.find_element(By.XPATH, '//table[@bgcolor="#ffffff"]')

In [ ]:
columns = [header.text for header in table.find_elements(By.XPATH, '//th/span')]
columns

In [ ]:
rows   = [row.find_elements(By.XPATH, './td/span') for row in table.find_elements(By.XPATH, '//tbody/tr')]
values = [[cell.text for cell in row] for row in rows if row]

In [ ]:
df = pd.DataFrame(values, columns=columns)[['Estado', 'Capital', 'Região']]
df.columns = ['estado', 'capital', 'regiao']
df = df.applymap(lambda x: x.title() if isinstance(x, str) else x)
df

In [ ]:
df_capital_populacao = pd.read_excel("PopulaçãoxCapital.xlsx")
df_capital_populacao.drop_duplicates(inplace=True)
df_capital_populacao[['capital', 'populacao']] = df_capital_populacao['Capital/populacao'].str.split(':', expand=True)
df_capital_populacao.drop(columns=['Capital/populacao'], inplace=True)
df_capital_populacao['populacao'] = df_capital_populacao['populacao'].str.replace(f"[{string.punctuation}]", "", regex=True).astype(int)
df_capital_populacao

In [ ]:
df_inner = pd.merge(df, df_capital_populacao, on='capital', how='inner')
df_inner